<table>
<tr>
<td width=15%><img src="./img/UGA.png"></img></td>
<td><center><h1>Introduction to Python for Data Sciences</h1></center></td>
<td width=15%><a href="https://tung-qle.github.io/" style="font-size: 16px; font-weight: bold">Quoc-Tung Le</a> </td>
</tr>
</table>


# 1 - Pytorch: a Numpy library that can differentiate

Pytorch is a Python library that are arguably the most popular for deep learning. It contains many similar functions that are implemented in Numpy (see the [second notebook](2_Numpy_and_co.ipynb)). Attention: many functions of these two libraries might have the same names, but their functionalities can be (entirely) different!

As we will see in this tutorial, in comparison to Numpy, Pytorch provides an additional important feature: Automatic Differentiation (AD, sometimes shortened to autodiff). That means if we implement a function using Pytorch library, we can compute its gradient with respect to its parameters efficiently. The gradient will be then used in the optimization algorithm (see Optimization course for more details).

The following code demonstrates the autodiff feature of Pytorch: in the following, we want to compute the gradient of the function:

$$f(x) = \frac{1}{2}x^\top \mathbf{A} x,$$

which are given by: $\nabla f(x) = \frac{1}{2}(\mathbf{A} + \mathbf{A}^\top)x$.

In [2]:

import torch

dim = 20
# Create a parameter x and a matrix A
x = torch.randn(dim, requires_grad=True)
A = torch.randn((dim, dim))

# Compute the function f(x) and assign to the variable y
y = 0.5 * torch.dot(x, torch.matmul(A, x))

# Differentiating the function f by calling y.backward()
y.backward()

# Accessing the gradient of f with respect to x
print(x.grad)
print(A.grad)

# Checking the calculation with the closed form gradient formula
try:
    torch.testing.assert_close(0.5 * torch.matmul(A + A.T,x), x.grad)
    print("Two vectors are equal. All is good")
except:
    print("Wrong calculation")


tensor([-1.3365, -1.3077, -2.2670, -5.7812,  0.7818,  1.3851,  1.8301,  4.1606,
        -1.6645, -2.7612, -0.6353,  0.2825, -1.7244,  3.4376,  1.7463,  2.1072,
         1.8103, -2.4540, -4.4065, -1.6249])
None
Two vectors are equal. All is good


## 1.1 - How to create Pytorch Tensors

There are many different methods to create a Pytorch tensor, either using Python list, numpy array or even randomization. They are shown in the following:

In [3]:
# Initialize a tensor using Python list
x = torch.tensor([[1.0, 2.0, 3.0], [-3.0, -2.0, -1.0]])
print(x)

tensor([[ 1.,  2.,  3.],
        [-3., -2., -1.]])


Unlike a Numpy array, a Pytorch tensor has many metadata fields. The following code shows how to access the information.

In [4]:
def show_info(x):
    print("Tensor info:")
    print(f"  shape        : {tuple(x.shape)}")
    print(f"  size         : {x.size()}")
    print(f"  dtype        : {x.dtype}")
    print(f"  device       : {x.device}")         # The output is either CPU or GPU, depending on the your implementation and hardware
    print(f"  requires_grad: {x.requires_grad}")  # If requires_grad = False, it is impossible to differentiate a function w.r.t. x. See the example with the quadratic function
    print(f"  is_leaf      : {x.is_leaf}")        # is_leaf = True if and only if requires_grad = False (convention) or it is initialized by the user and not a result of some operations
    print(f"  data         :\n{x}")    

show_info(x)

Tensor info:
  shape        : (2, 3)
  size         : torch.Size([2, 3])
  dtype        : torch.float32
  device       : cpu
  requires_grad: False
  is_leaf      : True
  data         :
tensor([[ 1.,  2.,  3.],
        [-3., -2., -1.]])


One can manually assign specific values for these metadata fields right at the moment or after creation.

Pay attention to the field _requires\_grad_ : it determines whether a variable can be differentiated or not. Therefore, when using Pytorch to perform optimization tasks, you need to ensure that _requires\_grad_ is __True__. Otherwise, the parameter will never be updated (we will see more about this in the exercise).

In [5]:
# We modify several metadata field of the same tensor x that we created previously

x.requires_grad = True
x = x.to(dtype = torch.float16)

show_info(x)
# We can also create a new tensor x, with desired metadata 
y = torch.tensor([[1, 2, 3], [-3, -2, -1]], requires_grad = True, dtype = torch.float16)
show_info(y)

Tensor info:
  shape        : (2, 3)
  size         : torch.Size([2, 3])
  dtype        : torch.float16
  device       : cpu
  requires_grad: True
  is_leaf      : False
  data         :
tensor([[ 1.,  2.,  3.],
        [-3., -2., -1.]], dtype=torch.float16, grad_fn=<ToCopyBackward0>)
Tensor info:
  shape        : (2, 3)
  size         : torch.Size([2, 3])
  dtype        : torch.float16
  device       : cpu
  requires_grad: True
  is_leaf      : True
  data         :
tensor([[ 1.,  2.,  3.],
        [-3., -2., -1.]], dtype=torch.float16, requires_grad=True)


One can use a Numpy array as the value of a newly-created Pytorch tensor as well

In [6]:
# Create a tensor using Numpy arrays

import numpy as np

x_array = np.reshape(np.arange(20), (4,5))
x = torch.Tensor(x_array)
print(x_array)
print(x)

[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]]
tensor([[ 0.,  1.,  2.,  3.,  4.],
        [ 5.,  6.,  7.,  8.,  9.],
        [10., 11., 12., 13., 14.],
        [15., 16., 17., 18., 19.]])


Finally, Pytorch offers several methods of creating random or special tensors

In [7]:
# Create a tensor by randomization

# Using uniform distribution
x = torch.rand((3,2))
print(x)
print(x.size())

# Using Gaussian distribution
x = torch.randn((3,2))
print(x)
print(x.size())

# Creat an identity, all-one and all-zero matrices
x_all_ones = torch.ones((3,2))
x_all_zeros = torch.zeros((3,2))
x_identity = torch.eye(3)  

print("All ones matrix:\n {}".format(x_all_ones))
print("All zeros matrix:\n {}".format(x_all_zeros))
print("Identity matrix:\n {}".format(x_identity))

tensor([[0.9572, 0.8288],
        [0.7195, 0.2063],
        [0.6423, 0.1076]])
torch.Size([3, 2])
tensor([[-0.3887, -1.5541],
        [-0.4626,  0.3839],
        [ 1.1924, -0.3491]])
torch.Size([3, 2])
All ones matrix:
 tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])
All zeros matrix:
 tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])
Identity matrix:
 tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])


## 1.2 - Operation on Pytorch tensors

Pytorch provides a plethora of tensor operations. For a complete presentation of its functionalities, you are advised to visit [this Pytorch documentation](https://docs.pytorch.org/docs/stable/torch.html). In the following, we will only introduce several frequent operations.

### 1.2.1 - Slicing, Joining and Mutating Operations

In [8]:
# Torch reshape
x = torch.tensor([i for i in range(100)])
print("Original tensor:\n {} \n".format(x))

# Reshape x to (10,10)
x_reshaped_1 = torch.reshape(x, (10, 10))
print("Reshape to a matrix 10 x 10:\n {}\n".format(x_reshaped_1))

# Reshape x to (2, 5, 5, 2)
x_reshaped_2 = torch.reshape(x, (2, 5, 5, 2))
print("Reshape to a tensor of dimensions 2 x 5 x 5 x 2: \n {}".format(x_reshaped_2))

# You can also obtain the same result using x_reshape_1

try:
    torch.testing.assert_close(x_reshaped_2, torch.reshape(x_reshaped_1, (2,5,5,2)))
    print("Same results")
except:
    print("Different results")

Original tensor:
 tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,
        72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89,
        90, 91, 92, 93, 94, 95, 96, 97, 98, 99]) 

Reshape to a matrix 10 x 10:
 tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
        [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
        [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
        [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
        [50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
        [60, 61, 62, 63, 64, 65, 66, 67, 68, 69],
        [70, 71, 72, 73, 74, 75, 76, 77, 78, 79],
        [80, 81, 82, 83, 84, 85, 86, 87, 88, 89],
        [90, 91, 92, 93, 94, 95, 96, 97, 98, 99]])


In [9]:
# Torch dimension transpose and permutation 
x = torch.arange(10).reshape((2,5))
print(x)

# For 2D tensor, transpose can be simply done by
x_transpose_1 = x.T
print("First method:\n {}".format(x_transpose_1))

# For general tensor with multiple dimension, use torch.transpose(your_tensor, dim0, dim1)
# dim0, dim1: two dimensions that you want to switch
x_transpose_2 = torch.transpose(x, 0, 1)
print("Second method:\n {}".format(x_transpose_2))

x_3d_tensor = torch.arange(30).reshape((2,3,5))
x_transpose_3 = torch.permute(x_3d_tensor, (1,2,0))
print(x_transpose_3.size())

tensor([[0, 1, 2, 3, 4],
        [5, 6, 7, 8, 9]])
First method:
 tensor([[0, 5],
        [1, 6],
        [2, 7],
        [3, 8],
        [4, 9]])
Second method:
 tensor([[0, 5],
        [1, 6],
        [2, 7],
        [3, 8],
        [4, 9]])
torch.Size([3, 5, 2])


In [10]:
# Torch concatenate
# Concatenate a matrix 2 x 5 with another of size 6 x 5 to get a 8 x 5 matrix
x = torch.arange(10).reshape((2,5))
y = torch.arange(30).reshape((6,5))
z = torch.concat([x,y], dim = 0)
print("Size of the first new concatenated tensor: {}".format(z.size()))

# Concatenate a matrix 5 x 2 with another of size 5 x 6 to get a 5 x 8 matrix
x = torch.arange(10).reshape((5,2))
y = torch.arange(30).reshape((5,6))
z = torch.concat([x,y], dim = 1)
print("Size of the second new concatenated tensor: {}".format(z.size()))

Size of the first new concatenated tensor: torch.Size([8, 5])
Size of the second new concatenated tensor: torch.Size([5, 8])


In [11]:
# Torch stack
# In comparison to torch concatenate, torch stack creates a new tensor with one more dimension

tensor_list = [torch.randn(3,2) for i in range(5)]
tensor_stacked_0 = torch.stack(tensor_list, dim = 0)
tensor_stacked_1 = torch.stack(tensor_list, dim = 1)
tensor_stacked_2 = torch.stack(tensor_list, dim = 2)

print(tensor_stacked_0.size())
print(tensor_stacked_1.size())
print(tensor_stacked_2.size())

torch.Size([5, 3, 2])
torch.Size([3, 5, 2])
torch.Size([3, 2, 5])


### 1.2.2 - Pointwise Operations

Similar to Numpy, Pytorch has many different function for pointwise operations, i.e., those of the forms:

$$\begin{pmatrix} x_{i_1\ldots i_n} \end{pmatrix}_{i_1,\ldots,i_n} \mapsto \begin{pmatrix} f(x_{i_1\ldots i_n}) \end{pmatrix}_{i_1,\ldots,i_n} \qquad \text{or} \qquad \begin{pmatrix} x_{i_1\ldots i_n} \end{pmatrix}_{i_1,\ldots,i_n} \times \begin{pmatrix} y_{i_1\ldots i_n} \end{pmatrix}_{i_1,\ldots,i_n} \mapsto \begin{pmatrix} g(x_{i_1\ldots i_n}, y_{i_1\ldots i_n}) \end{pmatrix}_{i_1,\ldots,i_n}$$

where $f: \mathbb{R} \to \mathbb{R}$ and $g: \mathbb{R} \times \mathbb{R} \to \mathbb{R}$.

In [12]:
x = torch.randn(3,5)

# f(x) = sin(x), cos(x), ReLU(x), |x|, exponent

print(x)
print(torch.sin(x))
print(torch.cos(x))
print(torch.relu(x))
print(torch.abs(x))
print(torch.exp(x))

tensor([[ 1.9236,  0.1250, -0.3960, -0.0285,  2.2730],
        [-0.6047, -0.1278,  1.1359,  0.6149,  2.2092],
        [ 0.5945, -0.0332, -3.0123, -1.6577, -1.3400]])
tensor([[ 0.9384,  0.1247, -0.3857, -0.0285,  0.7634],
        [-0.5685, -0.1274,  0.9069,  0.5769,  0.8031],
        [ 0.5601, -0.0332, -0.1290, -0.9962, -0.9735]])
tensor([[-0.3455,  0.9922,  0.9226,  0.9996, -0.6459],
        [ 0.8227,  0.9919,  0.4213,  0.8168, -0.5959],
        [ 0.8284,  0.9994, -0.9916, -0.0868,  0.2288]])
tensor([[1.9236, 0.1250, 0.0000, 0.0000, 2.2730],
        [0.0000, 0.0000, 1.1359, 0.6149, 2.2092],
        [0.5945, 0.0000, 0.0000, 0.0000, 0.0000]])
tensor([[1.9236, 0.1250, 0.3960, 0.0285, 2.2730],
        [0.6047, 0.1278, 1.1359, 0.6149, 2.2092],
        [0.5945, 0.0332, 3.0123, 1.6577, 1.3400]])
tensor([[6.8454, 1.1331, 0.6730, 0.9719, 9.7089],
        [0.5462, 0.8801, 3.1140, 1.8494, 9.1084],
        [1.8122, 0.9673, 0.0492, 0.1906, 0.2618]])


In [13]:
# Note that these previous functions do not change the value of x. In fact, they will compute the value f(x) and save it to a new memory allocation
# We can compute the function in-place by adding _ at the end of these functions

x = torch.randn(3,5)
print("Original value:\n {}".format(x))
torch.sin_(x)
print("New value:\n {}".format(x))

Original value:
 tensor([[-0.9731,  0.0520, -0.2603, -0.2928, -0.0179],
        [-0.7279,  2.0282,  1.4706, -2.1094, -0.2772],
        [ 1.1572, -1.3377,  0.8996, -0.4114,  0.2723]])
New value:
 tensor([[-0.8266,  0.0520, -0.2574, -0.2886, -0.0179],
        [-0.6653,  0.8972,  0.9950, -0.8584, -0.2737],
        [ 0.9157, -0.9730,  0.7831, -0.3999,  0.2689]])


In [14]:
x = torch.randn(3,5)
y = torch.randn(3,5)

# g(x, y) = x + y, x - y, x * y, x / y
print(x + y)
print(x - y)
print(x * y)
print(x / y)

tensor([[ 0.4831,  0.3268, -1.6203,  1.3995, -2.1340],
        [-1.0748,  0.0662, -0.9080,  2.0779, -0.5779],
        [ 1.6677,  0.6566, -0.5541, -2.9831, -2.0007]])
tensor([[-1.7067, -0.3064, -0.7179, -0.6240,  2.3520],
        [-0.8898, -1.8300,  0.3410,  0.9940,  2.8268],
        [ 0.1448,  0.9472, -2.1386, -3.2259,  0.6446]])
tensor([[-0.6698,  0.0032,  0.5275,  0.3923, -0.2445],
        [ 0.0909, -0.8362,  0.1771,  0.8325, -1.9142],
        [ 0.6901, -0.1165, -1.0666, -0.3769,  0.8968]])
tensor([[ -0.5587,   0.0322,   2.5910,   0.3833,  -0.0486],
        [ 10.6206,  -0.9302,   0.4540,   2.8340,  -0.6605],
        [  1.1902,  -5.5197,  -1.6994, -25.5746,   0.5127]])


In the previous example, noticing that $x$ and $y$ share the same shapes. However, it is also possible to perform operations for $x$ and $y$ of different shapes under certain conditions. Broadcasting allows to reduce memory footprint, simplify code and optimized performance.

Consider $x \in \mathbb{R}^{3 \times 5}$ and $y \in \mathbb{R}^{5}$. The value of $x + y$ will be given by $x + y^\star$ where $y^\star \in \mathbb{R}^{3 \times 5}$ is a matrix whose rows are all equal to $y$.

In [15]:
x = torch.randn(3,5)
y = torch.randn(5)

# Use broadcasting
print(x + y)

# More explicit code, but lengthy and not memory efficient because we need to explicitly compute y*
print(x + torch.stack([y for i in range(3)], dim = 0))

# If you want to add a certain vector to all columns, then do the following
z = torch.randn(3)

# First, reshape the vector to (3,1). Pytorch broadcasting will handle the rest
z = z.reshape((3,1))
print(x + z)

tensor([[ 1.1587, -2.3399, -0.7537,  1.6816, -2.1473],
        [-0.3771, -0.0312,  1.3968,  0.2348, -0.7198],
        [ 1.9391, -0.1018, -0.6971, -1.6533, -0.5395]])
tensor([[ 1.1587, -2.3399, -0.7537,  1.6816, -2.1473],
        [-0.3771, -0.0312,  1.3968,  0.2348, -0.7198],
        [ 1.9391, -0.1018, -0.6971, -1.6533, -0.5395]])
tensor([[ 1.8817, -1.4485, -0.4668,  1.7063, -0.5627],
        [-0.9677, -0.4534,  0.3701, -1.0541, -0.4488],
        [ 1.9010,  0.0284, -1.1714, -2.3898,  0.2839]])


### 1.2.3 - Other useful operations

Dot product between two tensors - $\mathtt{torch.dot}$ and tensor norm - $\mathtt{torch.norm}$

In [16]:
x = torch.randn(3)
y = torch.randn(3)

# torch.dot(x, y) 
dot_product = torch.dot(x,y)
print("Dot product between two tensors: {}".format(dot_product))

# torch.norm
# L2 norm (or the Euclidean distance to 0) of a vector
norm_l2 = torch.linalg.norm(x)

# For matrix, additional parameters can be specified so that we can compute different matrix norm
x = torch.randn(10, 9)

# Frobenius norm
fro_norm = torch.linalg.norm(x)
print("Frobenius norm: {}".format(fro_norm))

# Operator norm: op(A) = \max {\|Ax\|_2 | \|x\|_2 = 1}
op_norm = torch.linalg.norm(x, ord = 2)
print("Operator norm: {}".format(op_norm))

Dot product between two tensors: -1.6195298433303833
Frobenius norm: 10.435500144958496
Operator norm: 5.635004997253418


Matrix-matrix and matrix-vector multiplicaiton - $\mathtt{torch.matmul}$ and its batch version - $\mathtt{torch.bmm}$

In [34]:
x = torch.randn(3,5)
y = torch.randn(5,4)

# Compute the product using torch.matmul
print(torch.matmul(x,y))

# We can use @ instead
print(x @ y)

# torch.matmul can be used also to compute the product between a matrix and a vector (and we get a vector as the result)
z = torch.randn(5)
print(torch.matmul(x,z))

# torch.bmm can be used to compute matrix multiplication by batch

x = torch.randn(10,3,5) # A batch of matrices (a 3D tensor of size batch_size x dim_1 x dim_2)
y = torch.randn(10,5,4) # A batch of matrices (a 3D tensor of size batch_size x dim_2 x dim_3)
z = torch.bmm(x,y) # A batch of matrices (of size batch_size x dim_1 x dim_3)
print(z.size())

# One can compare the performance between torch.bmm and torch.mm when it comes to large batches of matrix multiplication
# In general, the larger is the batch size, the more efficient is torch.bmm in comparison to torch.mm 

import time

dim = 100
batch_size = 5000
x = torch.randn(batch_size, dim, dim)
y = torch.randn(batch_size, dim, dim)

start = time.perf_counter()

z = torch.zeros(batch_size, dim, dim)
for idx in range(len(x)):
    z[idx,:,:] = torch.matmul(x[idx,:,:], y[idx, :, :])

end = time.perf_counter()

print("Using loop takes {} seconds".format(end - start))

start = time.perf_counter()
z = torch.bmm(x,y)
end = time.perf_counter()

print("Using bmm takes {} seconds".format(end - start))

tensor([[ 0.5102, -5.0278,  2.4971,  0.6991],
        [-1.3725, -2.2848,  1.8377, -0.3113],
        [-0.0607,  2.3569, -6.4740, -2.7508]])
tensor([[ 0.5102, -5.0278,  2.4971,  0.6991],
        [-1.3725, -2.2848,  1.8377, -0.3113],
        [-0.0607,  2.3569, -6.4740, -2.7508]])
tensor([-1.9217, -0.3279, -0.9894])
torch.Size([10, 3, 4])
Using loop takes 0.2160515310001756 seconds
Using bmm takes 0.059521857000163436 seconds


Matrix inversion - $\mathtt{torch.inverse}$

In [18]:
x = torch.randn(3,3)
print(torch.inverse(x))

tensor([[-0.2798,  0.0622, -0.6481],
        [-0.3564,  0.1278,  0.4375],
        [-0.0744,  0.7727,  0.1693]])


Singular Value Decomposition: Given a matrix $\mathbf{A}$ (of arbitrary dimension), it can always be written as:

$$\mathbf{A} = \mathbf{U}\mathbf{S}\mathbf{V}^\top$$

where $\mathbf{U}, \mathbf{V}$ are orthogonal and $\mathbf{S}$ is a (possibly rectangular) diagonal matrix. Pytorch allows to compute such decomposition with $\mathtt{torch.linalg.svd}$

In [21]:
x = torch.randn(3,4)
u, s, v = torch.linalg.svd(x)
print(u)
print(s)
print(v)

tensor([[ 0.8889,  0.1393, -0.4365],
        [-0.4009,  0.6977, -0.5937],
        [-0.2218, -0.7027, -0.6760]])
tensor([1.5274, 1.0109, 0.2802])
tensor([[ 0.9686,  0.1166, -0.2186,  0.0198],
        [-0.0577,  0.5815, -0.0191, -0.8112],
        [-0.2405,  0.4011, -0.8224,  0.3240],
        [ 0.0245,  0.6981,  0.5249,  0.4863]])


## 1.3 - Compute the gradient of a function

Finally, we investiage how to compute the gradient of a function $f$ w.r.t. a variable $x$. For a concrete example, we consider the following function:

\begin{equation}
    f(\mathbf{U}, \mathbf{V}) = \|\mathbf{A} - \mathbf{U}\mathbf{V}^\top\|_F^2 \qquad \text{where} \qquad \mathbf{U} \in \mathbb{R}^{m \times r}, \mathbf{V} \in \mathbb{R}^{n \times r}, \mathbf{A} \in \mathbb{R}^{m \times n}
\end{equation}

We will compute the gradient w.r.t. $\mathbf{U}$ and $\mathbf{V}$ using the following code where $m = n = 10, r = 2$ (you can change these parameters).

In [131]:
m = 10
n = 10
r = 2

# Initialize u and v (using any method in the section 1.1)
A = torch.randn(m,n)

# Do not forget to set requires_grad = True for U and V (the variables that we want to differentiate w.r.t.)
U = torch.rand(m, r, requires_grad = True)
V = torch.rand(n, r, requires_grad = True)

From here, there are two main Pytorch functions that can be used for the automatic differentiation: $\mathtt{torch.grad.autograd()}$ and $\mathtt{backward()}$

In [132]:
# Using backward()

loss_fn = torch.linalg.norm(A - U @ V.T) ** 2
loss_fn.backward()

# Acessing the gradient of U and V using their field .grad

print("Gradient of U:\n {}".format(U.grad))
print("Gradient of V:\n {}".format(V.grad))

# Pay attention, if you backward twice, the gradient will be doubled
# In fact, the field .grad is accumulated: what you get is the sum of gradients of all previous backward() calls
loss_fn = torch.linalg.norm(A - U @ V.T) ** 2
loss_fn.backward()

# Acessing the gradient of U and V using their field .grad

print("Gradient of U after two backwards:\n {}".format(U.grad))
print("Gradient of V after two backwards:\n {}".format(V.grad))

Gradient of U:
 tensor([[ 7.9559,  6.0539],
        [ 4.1504,  7.5828],
        [ 2.5371,  2.4449],
        [ 1.8039,  3.5000],
        [ 6.1956,  4.1425],
        [11.0393, 15.0167],
        [ 6.4536,  4.7267],
        [ 6.1653,  7.8245],
        [ 2.6021,  0.4974],
        [ 3.1821,  5.8985]])
Gradient of V:
 tensor([[ 7.5575,  4.3580],
        [ 6.3633,  7.8859],
        [ 0.4309,  1.0458],
        [-4.6868, -5.2338],
        [ 6.7658,  7.0737],
        [ 4.1258,  6.5767],
        [-0.4851,  7.4583],
        [ 2.0126,  5.9917],
        [10.3318, -2.1440],
        [ 9.9368,  7.7034]])
Gradient of U after two backwards:
 tensor([[15.9118, 12.1078],
        [ 8.3009, 15.1656],
        [ 5.0741,  4.8898],
        [ 3.6078,  7.0000],
        [12.3912,  8.2849],
        [22.0786, 30.0333],
        [12.9071,  9.4533],
        [12.3307, 15.6491],
        [ 5.2042,  0.9948],
        [ 6.3641, 11.7971]])
Gradient of V after two backwards:
 tensor([[ 15.1151,   8.7161],
        [ 12.7265,  15.

In [67]:
# Using torch.grad.autograd(): you need to write a Python function that takes U and V as parameters and return f as the result

loss_fn = torch.linalg.norm(A - U @ V.T) ** 2

gradients = torch.autograd.grad(loss_fn, inputs = [U, V])

# torch.autograd.grad returns a list of gradients whose order is the same as the list of parameters that we provide for the argument inputs

print("Gradient of U:\n {}".format(gradients[0]))
print("Gradient of V:\n {}".format(gradients[1]))

Gradient of U:
 tensor([[ 7.8927,  3.0555],
        [ 3.4071,  0.8600],
        [10.9252,  8.3503],
        [ 9.4930,  2.0924],
        [12.5562,  8.7307],
        [ 4.9408,  3.9615],
        [ 6.5477, 12.6190],
        [ 7.6172,  6.3270],
        [12.2745, 12.2227],
        [ 8.4445,  6.1313]])
Gradient of V:
 tensor([[ 5.3973, 11.1532],
        [ 1.4212, -0.7133],
        [ 6.3601,  3.1715],
        [ 8.4189,  8.9069],
        [ 1.6081, 10.8099],
        [ 4.6290,  7.5384],
        [13.4344, 11.1798],
        [12.5221, 11.0223],
        [ 2.2346,  4.4744],
        [ 3.7380,  5.8154]])


In [68]:
# Differentiate w.r.t. the matrix A
A = torch.randn(m,n)

loss_fn = torch.linalg.norm(A - U @ V.T) ** 2
loss_fn.backward()

print(f"We will get {A.grad} because the field A.requires_grad = {A.requires_grad}")

# Now change it
A.requires_grad = True

# Remember to re-define loss_fn, otherwise, you cannot differentiate anymore
loss_fn = torch.linalg.norm(A - U @ V.T) ** 2
loss_fn.backward()

print("Now we get the gradient of A w.r.t. f: \n {}".format(A.grad))


We will get None because the field A.requires_grad = False
Now we get the gradient of A w.r.t. f: 
 tensor([[-1.4920,  1.1833, -0.3971, -0.5585,  3.0697,  0.2429,  0.4414, -4.4077,
         -3.6110,  0.7437],
        [-0.3568, -3.3341, -2.0782, -0.2510, -3.0174, -1.5164, -2.0860,  0.3596,
         -0.9220,  0.8905],
        [-4.0037, -1.5907, -0.5586, -3.2769, -0.0642,  1.4206, -3.8088, -2.4067,
         -0.0422,  0.3524],
        [-1.3092, -2.2984, -3.4772, -0.8878,  2.3446,  0.1379, -3.8952, -5.0607,
         -0.9229,  1.3610],
        [-2.1617, -0.7055, -3.9768, -3.9335, -3.5363, -5.3020, -1.0753, -1.8726,
         -1.0962, -1.3823],
        [ 0.9136,  1.4145, -1.8772, -1.2072, -1.0295, -1.2667, -1.2855, -4.2258,
         -0.8246,  0.1640],
        [-1.0351,  0.5156,  0.4445,  0.5564,  0.6262, -0.4407, -1.1818, -3.7739,
          2.7466, -3.2798],
        [-2.0172,  1.1386,  1.5795, -4.0885,  2.9253,  1.0934, -1.9789,  0.0205,
          0.4236, -2.0339],
        [-2.5574,  0.8024, -

# 2 - Several Pytorch modules for neural networks training

Pytorch provides users with many modules and built-in functions that are useful in both academic and industrial research and deployment of deep learning. Since our course provides only an overview of the main functionalities, we only deal with the vanilla deep learning architectures: the Multi-Layer Perceptrons (MLP)s and Pytorch components related to these architectures.

Given $\theta = \{(\mathbf{W_\ell}, b_\ell) \mid \ell = 1, \ldots, L\}$ a sequence of pairs of matrices and biases, the corresponding MLP encodes the following function:

\begin{equation}
    f_\theta (x) = \mathbf{W}_L \sigma (\mathbf{W}_{L-1} \sigma (\ldots \sigma(\mathbf{W}_1 x + b_1) \ldots ) + b_{L-1}) + b_L
\end{equation}

or equivalently, 
\begin{equation}
    f_\theta (x) = f_{\mathbf{W}_L, b_L} \circ \sigma \circ \ldots \sigma \circ f_{\mathbf{W_1}, b_1} (x)
\end{equation}

where $f_{\mathbf{W}_\ell, b_\ell}(x) = \mathbf{W}_\ell x + b_\ell$ is an affine function and $\sigma: \mathbb{R}^d \to \mathbb{R}^d$ is a pointwise function such as $\mathtt{torch.ReLU}$ or $\mathtt{torch.exp}, \ldots$ (see Section 1.2.2). You can use the built-in Pytorch functions for these components.

In [87]:
def print_model_params(model: torch.nn.Module):
    print(f"Model: {model.__class__.__name__}\n")
    
    print(f"{'Parameter':25} {'Shape':25} {'Requires_grad'}")
    print("-" * 70)
    
    for name, param in model.named_parameters():
        print(f"{name:25} {str(list(param.shape)):25} {param.requires_grad}")
    
    total_params = sum(p.numel() for p in model.parameters())
    print("\n" + "-" * 70)
    print(f"Total parameters: {total_params:,}")

In [95]:
# Linear layers 
dim_in = 10        # Input dimension
dim_out = 15       # Output dimension

linear_layer = torch.nn.Linear(dim_in, dim_out)

# Showing the size of the (weight) matrix W_l and the bias vector b_l
print_model_params(linear_layer)

# Compute the results given a vector input x
x = torch.randn(dim_in)
print(linear_layer(x))

# You can pass the option so that the linear layer does not have bias vector
linear_layer_without_bias = torch.nn.Linear(dim_in, dim_out, bias = False)
print("\n\n Models without bias:")
print_model_params(linear_layer_without_bias)

Model: Linear

Parameter                 Shape                     Requires_grad
----------------------------------------------------------------------
weight                    [15, 10]                  True
bias                      [15]                      True

----------------------------------------------------------------------
Total parameters: 165
tensor([-0.8236, -0.7631,  0.6164,  0.6084,  0.3192,  0.1023,  0.8852, -0.3812,
        -1.2504, -0.2486, -1.0723, -0.3228, -0.2335,  0.8313, -0.6257],
       grad_fn=<ViewBackward0>)


 Models without bias:
Model: Linear

Parameter                 Shape                     Requires_grad
----------------------------------------------------------------------
weight                    [15, 10]                  True

----------------------------------------------------------------------
Total parameters: 150


We can use $\mathtt{torch.nn.Sequential}$ modules to group several linear and non-linear layers together. For non-linear functions such as ReLU, we have to use $\mathtt{torch.nn.ReLU}$ instead of $\mathtt{torch.relu}$. The former is a module while the latter is a function. By design, $\mathtt{torch.nn.Sequential}$ takes a list of modules/classes, and not a function. Therefore, we cannot use $\mathtt{torch.nn.ReLU}$ and $\mathtt{torch.relu}$ interchangebly even if the two functions are semantically identical.

In [115]:
dim = 20

# Passing a series of modules and classes to torch.nn.Sequential to create a complex MLP
neural_network = torch.nn.Sequential(
    torch.nn.Linear(dim, dim),
    torch.nn.ReLU(),                   # Remember to use torch.nn.ReLU, instead of torch.relu
    torch.nn.Linear(dim, dim),
    torch.nn.ReLU(),
    torch.nn.Linear(dim, 1)
)

print_model_params(neural_network)

# If you want to name the modules in your neural networks so that you gain meaningful access later, you can use OrderedDict
from collections import OrderedDict

neural_network_named = torch.nn.Sequential(
    OrderedDict(
        [
            ("linear1", torch.nn.Linear(dim, dim)),
            ("relu1", torch.nn.ReLU()),
            ("linear2", torch.nn.Linear(dim, dim)),
            ("relu2", torch.nn.ReLU()),
            ("linear3", torch.nn.Linear(dim, 1))
        ]
    )
)
print_model_params(neural_network_named)

Model: Sequential

Parameter                 Shape                     Requires_grad
----------------------------------------------------------------------
0.weight                  [20, 20]                  True
0.bias                    [20]                      True
2.weight                  [20, 20]                  True
2.bias                    [20]                      True
4.weight                  [1, 20]                   True
4.bias                    [1]                       True

----------------------------------------------------------------------
Total parameters: 861
Model: Sequential

Parameter                 Shape                     Requires_grad
----------------------------------------------------------------------
linear1.weight            [20, 20]                  True
linear1.bias              [20]                      True
linear2.weight            [20, 20]                  True
linear2.bias              [20]                      True
linear3.weight          

In [120]:
# If submodules are named (using OrderedDict), you have their access in the neural networks as follows
print("Information of the first linear layers of my neural network")
print_model_params(neural_network_named.linear1)

# You can do the same without OrderedDict, but you need to remember the ordered of your submodules
print_model_params(neural_network[0])

Information of the first linear layers of my neural network
Model: Linear

Parameter                 Shape                     Requires_grad
----------------------------------------------------------------------
weight                    [20, 20]                  True
bias                      [20]                      True

----------------------------------------------------------------------
Total parameters: 420
Model: Linear

Parameter                 Shape                     Requires_grad
----------------------------------------------------------------------
weight                    [20, 20]                  True
bias                      [20]                      True

----------------------------------------------------------------------
Total parameters: 420


The second way to build complex neural network is to build a new class inheriting $\mathtt{nn.Module}$ (see [First lesson](1_Basics.ipynb) for a remind of classes in Python). This method allows a more flexible way to construct a neural network since they might not be just sequential.

In [133]:
import torch.nn as nn 
import torch.nn.functional as F

class MLP(nn.Module):
    # Init function: usually where we define parameters
    def __init__(self, dim_in, dim_out):
        super(MLP, self).__init__() 
        
        self.linear1 = nn.Linear(dim_in, dim_in)
        self.linear2 = nn.Linear(dim_in, dim_in)
        self.linear3 = nn.Linear(dim_in, dim_out)

        self.activation_func = F.relu

    # Forward function: where we define how to compute an output, given an input
    def forward(self, x):
        x = self.activation_func(self.linear1(x))
        x = self.activation_func(self.linear2(x))
        x = self.linear3(x)
        return x 

print_model_params(MLP(dim_in = dim, dim_out = 1))

# We can customize our model with a greater flexibility
# In the following, we will use linear 2 twice: after before linear 1 and after linear 1. This is impossible to do with Sequential

class MLP_customized(nn.Module):
    def __init__(self, dim_in, dim_out):
        super(MLP, self).__init__() 
        
        self.linear1 = nn.Linear(dim_in, dim_in)
        self.linear2 = nn.Linear(dim_in, dim_in)
        self.linear3 = nn.Linear(dim_in, dim_out)

        self.activation_func = F.relu

    def forward(self, x):
        x = self.activation_func(self.linear2(x))      # New line
        x = self.activation_func(self.linear1(x))
        x = self.activation_func(self.linear2(x))
        x = self.linear3(x)
        return x 

Model: MLP

Parameter                 Shape                     Requires_grad
----------------------------------------------------------------------
linear1.weight            [20, 20]                  True
linear1.bias              [20]                      True
linear2.weight            [20, 20]                  True
linear2.bias              [20]                      True
linear3.weight            [1, 20]                   True
linear3.bias              [1]                       True

----------------------------------------------------------------------
Total parameters: 861


To learn the model, we need to define a loss function and perform optimization. As most algorithm in neural network training require gradient, we can use our previous method to compute gradient of parameters. We consider the simplest loss functions - the quadratic loss on a single sample:

\begin{equation}
    \mathcal{L}(\theta) = (f_\theta(x) - y)^2
\end{equation}

In [141]:
# Learning samples
x = torch.randn(dim)
y = torch.randn(1)

# Models
neural_network = MLP(dim_in=dim, dim_out=1)

# Loss function
loss_fn = (neural_network(x) - y) ** 2

# Differentiation 
loss_fn.backward()

# Get the gradient of the weight matrix of the first linear layer
print(neural_network.linear1.weight.grad)

tensor([[ 5.5559e-03,  1.3487e-01,  6.5151e-02, -5.1109e-02, -3.9494e-02,
          8.7143e-02, -2.9409e-02,  1.0440e-01, -1.4997e-01, -1.0820e-01,
         -4.9237e-02, -1.6397e-02, -9.2151e-02,  5.7651e-02, -1.1397e-01,
         -1.1699e-01,  3.9388e-02, -3.2028e-02,  9.1441e-03,  7.2132e-02],
        [ 8.7015e-04,  2.1124e-02,  1.0204e-02, -8.0045e-03, -6.1854e-03,
          1.3648e-02, -4.6060e-03,  1.6351e-02, -2.3487e-02, -1.6946e-02,
         -7.7115e-03, -2.5681e-03, -1.4432e-02,  9.0292e-03, -1.7850e-02,
         -1.8323e-02,  6.1689e-03, -5.0161e-03,  1.4321e-03,  1.1297e-02],
        [-0.0000e+00, -0.0000e+00, -0.0000e+00,  0.0000e+00,  0.0000e+00,
         -0.0000e+00,  0.0000e+00, -0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00, -0.0000e+00,  0.0000e+00,
          0.0000e+00, -0.0000e+00,  0.0000e+00, -0.0000e+00, -0.0000e+00],
        [-0.0000e+00, -0.0000e+00, -0.0000e+00,  0.0000e+00,  0.0000e+00,
         -0.0000e+00,  0.0000e+00, 

# Exercises

1. Compute the maximum eigenvalue of a symmetric matrix using power method in Pytorch: Given a semi-positive definite matrix $\mathbf{A} \in \mathbb{R}^{d \times d}$, one can compute the largest eigenvalue of $\mathbf{A}$ as follows:
- Randomize a vector $v \in \mathbb{R}^{d \times d}$.
- Repeatedly perform the following operation: $v \gets \frac{\mathbf{A}v}{\|\mathbf{A}v\|}$ for $k$ sufficiently large number of iterations.
- Return $\|v\|$.

In [143]:
def your_function(A: torch.Tensor, k: int):
    """
        Implement your power method here
    """
    return 

In [150]:
# Test the correctness of your method
X = torch.randn(10,10)
A = torch.matmul(X,X.T)    # A is semi-positive definite

your_max_eigs = your_function(A, k = 20)
true_max_eigs = max(torch.linalg.eigvals(A).real)

print(your_max_eigs)
print(true_max_eigs)

None
tensor(32.4726)


2. Compute the best rank-$r$ approximation of a given matrix: Given a matrix $\mathbf{A} \in \mathbb{R}^{m \times n}$, we would like to find a matrix $\mathbf{A} \in \mathbb{R}^{m \times n}$ whose rank is at most $r$ that minimizes $\|\mathbf{A} - \mathbf{A}'\|_F^2$. Mathematically, it is formulate as:
\begin{equation}
\text{Minimze} \|\mathbf{A} - \mathbf{A}'\|_F^2 \qquad \text{such that} \qquad \mathtt{rank}(\mathbf{A}') \leq r.
\end{equation}

The analytic solution of this problem is given by:

$$\mathbf{A}' = \mathbf{U}[:,r] \mathtt{diag}(\lambda_1, \ldots, \lambda_r) \mathbf{V}[:,r]^\top$$

where $\mathbf{U}, \mathbf{V}$ are two orthogonal matrices in the SVD of $\mathbf{A}$ and $\lambda_1 \geq \ldots \geq \lambda_r$ are the $r$ largest singular values of $\mathbf{A}$ (also given by the SVD, see Section 1.2).

Write a function using Pytorch to find $\mathbf{A}'$ given $\mathbf{A}$.

In [ ]:
def best_rank_approximation(A: torch.Tensor, r: int):
    """Write your code here"""
    return

3. Write a gradient descent for the neural network training problem: Consider optimizing the following function:
\begin{equation}
f(\mathbf{W}_1, \mathbf{W}_2, b_1, b_2) = (y - \mathbf{W}_2\sigma(\mathbf{W}_1 x + b_1) + b_2)^2,
\end{equation}
where $\sigma(x) = \frac{1}{1 + \exp(x)} $ applied in a coordinate-wise fashion (you can use $\mathtt{torch.sigmoid}$). Once compute its gradient, use the gradient descent formula to update your parameters:
\begin{equation}
    \theta_{k+1} = \theta_k - \alpha \nabla f(\theta_k).
\end{equation}

Attention: If you use $\mathtt{backward()}$ to compute the gradient, make sure to reset them after every iteration by the command: $\mathtt{your\_params.grad.zero\_()}$. Otherwise, the gradient will be accumulated.